In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_augmented_images_with_masks(image_dir, augmented_dir, target_size=(128, 128), num_classes=21):
    augmented_files = os.listdir(augmented_dir)
    base_names = {f.split('-')[0] for f in augmented_files}
    
    images = []
    masks = []
    
    for img_file in sorted(os.listdir(image_dir)):
        base_name = os.path.splitext(img_file)[0]
        
        if base_name in base_names:
            img = load_img(os.path.join(image_dir, img_file), target_size=target_size)
            img = img_to_array(img) / 255.0
            images.append(img)
            
            mask_path = os.path.join(image_dir, img_file)
            mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
            mask = img_to_array(mask).astype(np.int32)
            mask = np.squeeze(mask, axis=-1)
            
            mask[mask > 20] = 0
            
            mask = tf.keras.utils.to_categorical(mask, num_classes=num_classes)
            masks.append(mask)
    
    return np.array(images), np.array(masks)

image_dir = '../data/VOCdevkit/VOC2012/SegmentationClass'
augmented_dir = '../data/VOCdevkit/VOC2012/AugmentedImages'

images, masks = load_augmented_images_with_masks(image_dir, augmented_dir)
print(f"Loaded {len(images)} images and {len(masks)} masks from {image_dir} with corresponding augmented images.")


Loaded 47 images and 47 masks from ../data/VOCdevkit/VOC2012/SegmentationClass with corresponding augmented images.
